In [1]:
from torch.optim import Adam, SGD
import torch
import numpy as np
import ot
import matplotlib.pyplot as plt
from tqdm import tqdm
from Util_mixture import *
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def train(testing_ys, mat_dim, z_dim):
    num_layers, sub_net_size = 7, 64
    model = create_INN(num_layers, sub_net_size,dimension=mat_dim**2,dimension_condition=int(mat_dim**2/4))
    lr = 1e-4
    optimizer = Adam(model.parameters(), lr=lr)
    num_epochs = 400
    batch_size = 128
    num_samples_per_epoch = 1024
    optimizer = Adam(model.parameters(), lr = 1e-4)
    num_epochs = 100    
    batch_size = 64    
    num_samples_per_epoch = 2048

    prog_bar = tqdm(total=num_epochs)
    for i in range(num_epochs):
        data_loader = get_epoch_data_loader_new(num_samples_per_epoch, batch_size, mat_dim)
        loss = train_inn_epoch_mat(optimizer, model, data_loader, mat_dim)
        prog_bar.set_description('loss: {:.4f}'.format(loss))
        prog_bar.update()
    prog_bar.close()

    return model

In [5]:
def eval(testing_ys, mat_dim, z_dim):
    result = []
    testing_x_per_y = 2048
    
    weights1, weights2 = np.ones((testing_x_per_y,)) / testing_x_per_y, np.ones((testing_x_per_y,)) / testing_x_per_y
    weights1 = weights1.astype(np.float64)
    weights2 = weights2.astype(np.float64)
    w1 = 0.

    testing_num_y = len(testing_ys)
    prog_bar = tqdm(total=testing_num_y)
    prog_bar.set_description('Computing Wasserstein distances')
    for i, y in enumerate(testing_ys):
        true_posterior_samples = draw_posterior_matrix(y, mat_dim, testing_x_per_y)
        inflated_ys = y.flatten()[None, :].repeat(testing_x_per_y, 1)
        true_post = true_posterior_samples.reshape(-1, mat_dim ** 2)
        noise1 = torch.randn(testing_x_per_y, z_dim).to(device)
        rtarget = inflated_ys
        samples1 = model(torch.randn(testing_x_per_y, mat_dim**2, device=device), c= inflated_ys)[0].detach().cpu().numpy()
        samples1 = samples1.reshape(testing_x_per_y, mat_dim* mat_dim)
        result += [samples1]

        M1 = ot.dist(samples1, true_post)
        w1 += ot.emd2(weights1, weights2, M1)
        prog_bar.set_description('W: {:.3f}'.format(w1 / (i + 1)))
        prog_bar.update()
    return result

In [6]:
result_list = []
testing_num_y = 100
mat_dim = 8
z_dim = mat_dim


In [8]:
testing_xs = draw_x_mats(100, mat_dim)                
testing_ys = forward_mat(testing_xs)

In [9]:

model = train(testing_ys, mat_dim, z_dim)


loss: -29.3082: 100%|██████████| 100/100 [00:55<00:00,  1.81it/s]


In [10]:
result = eval(testing_ys, mat_dim, z_dim)


Computing Wasserstein distances:   0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([100, 8, 8]) torch.Size([100, 4, 4])
4
samples: (2048, 64), true: torch.Size([2048, 64])


W: 14.466:   1%|          | 1/100 [00:02<03:37,  2.20s/it]              

samples: (2048, 64), true: torch.Size([2048, 64])


W: 14.826:   2%|▏         | 2/100 [00:04<03:33,  2.17s/it]

...

samples: (2048, 64), true: torch.Size([2048, 64])


W: 15.350:  99%|█████████▉| 99/100 [03:40<00:02,  2.35s/it]

samples: (2048, 64), true: torch.Size([2048, 64])


W: 15.342: 100%|██████████| 100/100 [03:42<00:00,  2.31s/it]

[array([[-0.49589783,  0.61328113,  0.6305553 , ...,  0.7592481 ,
         1.4477004 ,  0.8806828 ],
       [ 0.5551277 , -0.14914364,  1.531405  , ...,  1.069781  ,
         0.5428651 ,  0.78692555],
       [ 1.0524828 ,  0.903377  ,  0.25060523, ...,  1.2989255 ,
         1.0647309 , -0.22277427],
       ...,
       [-0.33901373,  0.58017653,  1.149852  , ...,  0.9020556 ,
         0.8783412 ,  0.7507312 ],
       [-0.1299917 ,  0.5575352 ,  1.1343086 , ...,  0.8882068 ,
         0.7960025 ,  0.7952117 ],
       [-0.08245399,  0.28573114,  0.5146859 , ...,  0.49233532,
         0.8182048 ,  1.3119425 ]], dtype=float32), array([[ 1.0502943 ,  0.9162462 ,  0.54398346, ...,  0.7479236 ,
         1.1868901 ,  0.72786516],
       [-0.01978268,  0.1924966 ,  0.52045643, ...,  0.49247065,
         0.2107952 , -0.5426641 ],
       [ 0.37672612,  0.5686444 ,  0.58832467, ...,  0.10252452,
         1.0972012 ,  0.7217022 ],
       ...,
       [-0.41554093,  0.21634758,  0.20076889, ...,  1.018

In [28]:
models = create_INN(7, 64,dimension=mat_dim**2,dimension_condition=int(mat_dim**2/4))
sum(p.numel() for p in models.parameters() if p.requires_grad)#

W: 15.645: 100%|██████████| 100/100 [2:19:54<00:00, 83.94s/it]


160384